<a href="https://colab.research.google.com/github/iypc-team/CoLab/blob/master/Github_Repos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# %pip install GitPython

In [2]:
from __future__ import absolute_import

import git, os, pathlib, sys
from os.path import *

from google.colab import drive, files
drive.mount("/gdrive", force_remount=True)
%cd "/gdrive/My Drive"
%pwd

import BashColors
from BashColors import C
import FileUtilities
from FileUtilities import *
fu.listFiles(silent=True)

Mounted at /gdrive
/gdrive/My Drive
BashColors() Ok


In [3]:
%%writefile GithubRepos.py
# 09/08/2020
import git, os
from os.path import *
from BashColors import C

class GithubRepos:
        
    def __init__(self):
        self.gr = GithubRepos
        self.repoList = []
        
    def getCurrentProjectRepo(self):
        currentProjectURL = 'https://github.com/iypc-team/CurrentProject'
        
        localURL = os.path.join(os.curdir, basename(currentProjectURL))
        print(basename(currentProjectURL))
        if not exists(localURL):
            currentProjectRepo = git.Repo.clone_from(currentProjectURL, localURL)
            self.repoList.append(currentProjectRepo)
        else: print(f'{C.HEADER}{basename(localURL)} already exists')
    
    def getCoLabRepo(self):
        thisRepo = None
        colabURL = 'https://github.com/iypc-team/CoLab'
        self.updateRepoList(name=colabURL)
        localURL = os.path.join(os.path.curdir, basename(colabURL))
        if not exists(localURL):
            thisRepo = git.Repo.clone_from(colabURL, localURL)
            self.repoList.append(thisRepo)
            print(f'{C.Green}{thisRepo}')
        else: print(f'{C.Red}{thisRepo}')

    def listRemotes(self):
        print(f'{C.Blue}Remotes:')
        for rpo in self.repoList:
            print(rpo)
            print(f'{rpo}')

    def updateRepoList(self, name):
        if not name in self.repoList:
            self.repoList.append(name)
            
gr = GithubRepos()

Overwriting GithubRepos.py


In [4]:
import GithubRepos
from GithubRepos import *

tzi.getCurrentTimeIn(tzi.centralTZone, silent=False)

2020-09-08 08:21:51.227288-05:00
Sep/08/2020   8:21AM


In [5]:
fu.listFiles(silent=False)

/gdrive/My Drive
1. BashColors.py
Aug/30/2020   2:05PM    size: 1735 bytes
2. Bash_Colors.ipynb
Aug/21/2020   4:03PM    size: 4238 bytes

/gdrive/My Drive/CoLab
3. 00_DQN_Tutorial.ipynb
Sep/07/2020   6:16PM    size: 32711 bytes
4. AccessGoogleDrive.ipynb
Sep/07/2020   6:16PM    size: 5184 bytes
5. Accessing_GitHub.ipynb
Sep/07/2020   6:16PM    size: 3309 bytes
6. Accessing_files.ipynb
Sep/07/2020   6:16PM    size: 8618 bytes
7. Bash_Colors.ipynb
Sep/07/2020   6:16PM    size: 9977 bytes
8. BasicComputerVision.ipynb
Sep/07/2020   6:16PM    size: 4235 bytes
9. CartPole_Krohn.ipynb
Sep/07/2020   6:16PM    size: 4404 bytes
10. DDQN_Keras.ipynb
Sep/07/2020   6:16PM    size: 9246 bytes
11. DQN_Tutorial.ipynb
Sep/07/2020   6:16PM    size: 35453 bytes
12. DQN_with_GPU.ipynb
Sep/07/2020   6:16PM    size: 5623 bytes
13. Drive.ipynb
Sep/07/2020   6:16PM    size: 7371 bytes
14. DuelingDeepQNetwork.ipynb
Sep/07/2020   6:16PM    size: 6418 bytes
15. Dueling_ddqn_torch.ipynb
Sep/07/2020   6:16PM    si

In [6]:

for item in fu.allFilesDict:
    pth = fu.allFilesDict[item]
    realPth = realpath(pth)
    if pth.name.__contains__('GDriveFileUtilities'):
        print(item, realPth)
        

112 /gdrive/My Drive/PythonFiles/GDriveFileUtilities.py
117 /gdrive/My Drive/Swift/GDriveFileUtilities.py


In [12]:
%cd "/gdrive/My Drive/Swift"
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# List .txt files in the root.
#
# Search query reference:
# https://developers.google.com/drive/v2/web/search-parameters
listed = drive.ListFile({'q': "title contains '*.*' and 'root' in parents"}).GetList()
print(len(listed))
for file in listed:
    print('title {}, id {}'.format(file['title'], file['id']))

/gdrive/My Drive/Swift
14
title GithubRepos.py, id 1amWEVqGDNtx0saJlYw9oP8aWQDbIAtgI
title TZInfo.py, id 1bPKxKGdMJH0ZNxypirZ5oFzq3Bo5bac5
title FileUtilities.py, id 1Vb4BtGfE1e2rl9IiLIG59prFJDw5dl8G
title CurrentProject, id 14_AktnGoqDJvjGvTZdooufWaR53nmq1m
title CoLab, id 1xv4ODqH_JKNiDK7WymVQPfQ9gtkv41V6
title PhilTabor, id 1u_gH6P3uHh7X5bX4e4q1fIAmf9c_qnA2
title BashColors.py, id 1fzYKMsEsjvYP9orVJgzN9caMFg7TWEQk
title Bash_Colors.ipynb, id 1JiyPq4XEE1fsk0idqBRdf52M0M6oCMVl
title PythonFiles, id 1lT4BXbrPWKY_pBV4PrWeJHJvAjz8a0zv
title File_Utilities_2.ipynb, id 1ALtWbZHMJL-02WsuUazD0YbVxdRLCcBl
title Colab Notebooks, id 1peTs5vCXHTHgb_6QrKgj2NHTH1x4VCv7
title __pycache__, id 134fiKXtddCL6GTp5eKHFiJLI4HciGRZl
title Images, id 19hE4JAlcj7TxdrdAqfhOQSuV1i9YZYbE
title Swift, id 1-ZqppCTessl49Jp9dZQIwvJoXFoF4Vx2
